In [12]:
import sympy as sp

from symqups import s
from symqups.objects.base import PhaseSpaceObject, qpTypePSO, alphaTypePSO
from symqups.objects.scalars import q, p, alpha, alphaD, W, mu, _Primed, _DerivativeSymbol, hbar, _DePrimed, StateFunction
from symqups.objects.operators import qOp, pOp, annihilateOp, createOp, rho, Operator
from symqups.utils.algebra import define, qp2a, get_random_poly
from symqups.operations.operator_ordering import sOrdering, _make_normal_ordered
# from symqups.operations.quantization import s_quantize, naive_quantize
from symqups.objects import scalars
from symqups.objects.cache import _sub_cache
from symqups.operations.star_product import Bopp, Star, _replace_diff
from symqups.utils.multiprocessing import _mp_helper
from symqups.utils._internal._operator_handling import _separate_operator
from symqups.utils._internal._operator_handling import (
    _collect_alpha_type_oper_from_monomial, _separate_operator, _separate_by_oper_polynomiality
)


a = alpha()
a_1 = alpha(1)
aop = annihilateOp()
aop_1 = annihilateOp(1)
ad = alphaD()
ad_1 = alphaD(1)
adop = createOp()
adop_1 = createOp(1)
x = sp.Symbol("x")
qq = q()
pp = p()
W = scalars.W
s.val = 0
s

s

In [13]:
sOrdering(sOrdering(aop*adop)).has(Operator)

True

In [ ]:


_, a, b = _collect_alpha_type_oper_from_monomial(_separate_by_oper_polynomiality(_separate_operator(qp2a(qOp()).args[0])[1])[0])

_make_normal_ordered(a,b)

1

In [12]:
_collect_alpha_type_oper_from_monomial(sp.Symbol("x"))

(1,
 {: [\hat{a}^{\dagger}_{}, 0], 1: [\hat{a}^{\dagger}_{1}, 0]},
 {: [\hat{a}_{}, 0], 1: [\hat{a}_{1}, 0]})